In [5]:
from glob import glob
from h5py import File as h5File
import os
import numpy as np

import cv2 as cv
from tqdm import tqdm

import matplotlib.pyplot as plt

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Methods

In [3]:
def plot_img(img,x,y):
    plt.figure(figsize=(12,12))
    plt.imshow(
        img,
        vmin=0, vmax=255,
        cmap='gray'
    )
    plt.scatter([x],[y])
    plt.show()

# Load Data

In [4]:
IMG_OUTPUT_DIM = 128
if torch.cuda.is_available():
    DEVICE = 'cuda:0'
else:
    DEVICE = 'cpu'

In [6]:
if 'file' in locals():
    file.close()
file = h5File(
    "../data/raw/20230414_N2_pharyngeal_and_pumping/000003.h5"
)
data = file['data']
data.shape

(3600, 512, 512)

In [ ]:
if 'data_annotated' not in locals():
    data_annotated = dict()

In [ ]:
# t=0, i,j = 374, 289
# t=1000, i,j = 308, 181

In [ ]:
t = 0
i, j = 374, 289
img = data[t].copy()
img_annotated = np.zeros_like( img, dtype=np.float32 )
img_annotated[i,j] = 100.0
img_annotated = cv.GaussianBlur(img_annotated,(11,11), 0)
img_annotated = cv.resize(img_annotated, (IMG_OUTPUT_DIM, IMG_OUTPUT_DIM))
img_annotated /= img_annotated.max()
data_annotated[t] = img_annotated

##############################################################################

t = 1000
i, j = 308, 181
img = data[t].copy()
img_annotated = np.zeros_like( img, dtype=np.float32 )
img_annotated[i,j] = 100.0
img_annotated = cv.GaussianBlur(img_annotated,(11,11), 0)
img_annotated = cv.resize(img_annotated, (IMG_OUTPUT_DIM, IMG_OUTPUT_DIM))
img_annotated /= img_annotated.max()

data_annotated[t] = img_annotated

In [ ]:
plot_img(
    data[t], j,i
)

In [ ]:
# plt.figure(figsize=(12,12))
# plt.imshow(img[350:400, 250:300], cmap='gray')
# plt.show()

# plt.figure(figsize=(12,12))
# plt.imshow(img_annotated[350:400, 250:300])
# plt.show()

# X and Y

In [ ]:
ts = sorted(data_annotated)
X = np.array([
    data[t] for t in ts
])
y = np.array([
    data_annotated[t] for t in ts
])

In [ ]:
# for _x, _y in zip( X, y ):
#     plt.figure()
#     plt.imshow(_x)
#     plt.show()
#     plt.figure()
#     plt.imshow(_y)
#     plt.show()
#     print("\n"*5)

In [ ]:
X.shape, y.shape

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutions
        self.conv1 = nn.Conv2d(1, 4, 25)
        self.conv1_activation = nn.ReLU()
        self.conv1_pooling = nn.MaxPool2d(5, stride=2)
        # TODO: add max_pooling layers
        # https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d
        # Flatten
        self.flatten = nn.Flatten()
        # Denses
        self.linear1 = nn.Linear(
            in_features=234256,
            out_features=64
        )
        self.linear1_activation = nn.ReLU()
        self.dense = nn.Linear(
            in_features=64,
            out_features=IMG_OUTPUT_DIM**2
        )
        self.to_probability = nn.Sigmoid()
        return

    def forward(self, x):
        # Convolutions
        x = self.conv1_activation(self.conv1(x))
        x = self.conv1_pooling(x)
        # Flattern
        x = self.flatten(x)
        # Dense
        x = self.linear1_activation(
            self.linear1(x)
        )
        x = self.dense(x)
        return self.to_probability(x)

In [ ]:
model = Model().to(DEVICE)
model

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
X_train = torch.tensor(X[:, None, :, :], dtype=torch.float).to(DEVICE)
y_train = torch.tensor(
    np.reshape(y, (2,IMG_OUTPUT_DIM**2)),
    dtype=torch.float
).to(DEVICE)

In [ ]:
iterable = tqdm( range(1000), desc=f'Loss: {0.0:>7.3f}' )

for i_epoch in iterable:
    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    y_train_pred = model(X_train)

    # Compute the loss and its gradients
    loss = loss_fn(y_train_pred, y_train)
    loss.backward()

    # Adjust learning weights
    optimizer.step()

    # Report
    iterable.set_description(
        'Loss: {:>7.3f}'.format(
            loss.cpu().item()
        )
    )

In [ ]:
plt.figure()
plt.imshow(
    np.reshape(
        y_train_pred[1].detach().cpu().numpy(),
        (IMG_OUTPUT_DIM,IMG_OUTPUT_DIM)
    )
)
plt.show()
#########################################
plt.figure()
plt.imshow(
    np.reshape(
        y_train[1].cpu().numpy(),
        (IMG_OUTPUT_DIM,IMG_OUTPUT_DIM)
    )
)
plt.show()

In [ ]:
img_pred = np.reshape(
    y_train_pred[0].detach().cpu().numpy(),
    (IMG_OUTPUT_DIM,IMG_OUTPUT_DIM)
)

In [ ]:
plt.imshow(
    img_pred > 0.9
)

In [ ]:
np.where(img_pred > 0.9)

In [ ]:
np.where(
    data_annotated[0] == 1.0
)

In [ ]:
np.where(
    data_annotated[1000] == 1.0
)

In [ ]:
import time

In [ ]:
_start = time.time()
_N = 1000
with torch.no_grad():
    for _ in range(_N):
        # Make predictions for this batch
        y_train_pred = model(X_train)
#
_end = time.time()
_duration = _end - _start

In [ ]:
1000*(_duration / _N)

In [ ]:
x = torch.tensor(X[0][None,None,:,:], dtype=torch.float)

In [ ]:
l = nn.Conv2d(1, 5, 5)

In [ ]:
tmp = l(x)

In [ ]:
tmp.shape

In [ ]:
nn.Flatten()(tmp).shape

In [ ]:
512*512

In [ ]:
nn.MaxPool2d(5, stride=2)(tmp).shape

In [ ]:
5*252*252

In [ ]:
317520*262144